# Iron Condor Backtest with DoltHub Dataset

This notebook demonstrates a complete backtest of an Iron Condor strategy using real options data from DoltHub.

## Strategy Overview

**Iron Condor** is a neutral options strategy that profits from low volatility:
- Sell OTM put spread (collect premium)
- Sell OTM call spread (collect premium)
- Profit if underlying stays within the strikes until expiration
- Limited risk and limited reward

In [ ]:
# Imports
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add backtester to path
sys.path.insert(0, str(Path.cwd().parent))

from backtester import (
    DoltHubAdapter,
    MarketDataLoader,
    IronCondorStrategy,
    BacktestEngine,
    BacktestConfig,
    BlackScholesModel,
    PerformanceMetrics,
    PnLAttributionEngine,
    VisualizationEngine
)

pd.set_option('display.max_columns', None)
%matplotlib inline

## Step 1: Load Market Data from DoltHub

In [ ]:
# Configuration
DB_PATH = "/Users/janussuk/Desktop/dolt_data/options"
TICKER = "SPY"
START_DATE = "2024-01-01"
END_DATE = "2024-03-31"

print(f"Loading market data for {TICKER}...\n")

# Create adapter and loader
adapter = DoltHubAdapter(DB_PATH)
loader = MarketDataLoader(adapter)

# Load complete market data
market_data = loader.load(
    ticker=TICKER,
    start_date=START_DATE,
    end_date=END_DATE,
    build_vol_surface=True
)

print(f"\n✓ Market data loaded successfully!")

## Step 2: Design the Iron Condor Strategy

We'll construct an iron condor with:
- 45 days to expiration
- Strikes at ±5% and ±10% from spot

In [ ]:
# Strategy entry date
entry_date = pd.Timestamp("2024-01-03")
spot = market_data.get_spot(entry_date)

print(f"Entry Date: {entry_date.date()}")
print(f"Spot Price: ${spot:.2f}")

# Get available expirations
available_exps = loader.get_available_expirations(TICKER, entry_date.strftime('%Y-%m-%d'))
print(f"\nAvailable Expirations:")
for i, exp in enumerate(available_exps[:5]):
    days = (exp - entry_date).days
    print(f"  {i+1}. {exp.date()} ({days} days)")

# Select expiration ~45 days out
target_days = 45
expiry = min(available_exps, key=lambda x: abs((x - entry_date).days - target_days))
days_to_exp = (expiry - entry_date).days

print(f"\nSelected Expiration: {expiry.date()} ({days_to_exp} days)")

In [ ]:
# Define iron condor strikes
put_lower_strike = spot * 0.90   # Long put (10% OTM)
put_upper_strike = spot * 0.95   # Short put (5% OTM)
call_lower_strike = spot * 1.05  # Short call (5% OTM)
call_upper_strike = spot * 1.10  # Long call (10% OTM)

print(f"\nIron Condor Structure:")
print(f"  Put Spread:  LONG  ${put_lower_strike:.2f} / SHORT ${put_upper_strike:.2f}")
print(f"  Call Spread: SHORT ${call_lower_strike:.2f} / LONG  ${call_upper_strike:.2f}")
print(f"  Profit Zone: ${put_upper_strike:.2f} to ${call_lower_strike:.2f}")
print(f"  Width: {((call_lower_strike - put_upper_strike) / spot * 100):.1f}%")

# Create the strategy
iron_condor = IronCondorStrategy(
    underlying=TICKER,
    put_lower_strike=put_lower_strike,
    put_upper_strike=put_upper_strike,
    call_lower_strike=call_lower_strike,
    call_upper_strike=call_upper_strike,
    expiry=expiry,
    quantity=1.0
)

print(f"\n{iron_condor}")

## Step 3: Visualize the Risk Profile

In [ ]:
# Create visualization engine
viz = VisualizationEngine(use_plotly=False)

# Plot risk profile
model = BlackScholesModel(use_market_iv=True)
viz.plot_risk_profile(
    iron_condor,
    entry_date,
    market_data,
    model,
    title=f"Iron Condor Risk Profile - {TICKER}"
)

## Step 4: Configure and Run Backtest

In [ ]:
# Backtest configuration
config = BacktestConfig(
    start_date=entry_date,
    end_date=expiry,
    initial_capital=100000.0,
    transaction_cost_per_contract=0.65,
    transaction_cost_pct=0.0001,
    model=BlackScholesModel(use_market_iv=True)
)

# Create backtest engine
engine = BacktestEngine(market_data, config)

# Add strategy
engine.add_strategy(iron_condor, entry_date=entry_date)

# Run backtest
print("\nRunning backtest...\n")
results = engine.run()

## Step 5: Analyze Performance

In [ ]:
# Calculate metrics
metrics = PerformanceMetrics(results, risk_free_rate=0.05)
metrics.print_summary()

In [ ]:
# P&L Attribution
attribution = PnLAttributionEngine(results)
attribution.print_summary()

## Step 6: Visualizations

In [ ]:
# Equity curve
viz.plot_equity_curve(results, title="Iron Condor - Portfolio Value")

In [ ]:
# Greeks evolution
viz.plot_greeks(results, greeks=['delta', 'theta', 'vega'])

In [ ]:
# P&L Attribution
viz.plot_pnl_attribution(results, cumulative=True)

In [ ]:
# Drawdown chart
viz.plot_drawdown(results)

## Step 7: Detailed Analysis

In [ ]:
# Show daily results
print("Daily P&L Breakdown (first 10 days):\n")
results[['spot', 'portfolio_value', 'daily_pnl', 'delta', 'theta', 'vega']].head(10)

In [ ]:
# Spot price movement during the trade
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Spot price
ax1.plot(results.index, results['spot'], 'b-', linewidth=2, label='Spot Price')
ax1.axhline(y=put_upper_strike, color='r', linestyle='--', alpha=0.5, label='Short Put Strike')
ax1.axhline(y=call_lower_strike, color='g', linestyle='--', alpha=0.5, label='Short Call Strike')
ax1.axhline(y=spot, color='gray', linestyle=':', alpha=0.5, label='Entry Spot')
ax1.fill_between(results.index, put_upper_strike, call_lower_strike, alpha=0.1, color='green', label='Profit Zone')
ax1.set_ylabel('Price ($)')
ax1.set_title(f'{TICKER} Price Movement')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Portfolio value
ax2.plot(results.index, results['portfolio_value'], 'darkblue', linewidth=2)
ax2.axhline(y=config.initial_capital, color='gray', linestyle='--', alpha=0.5, label='Initial Capital')
ax2.set_xlabel('Date')
ax2.set_ylabel('Portfolio Value ($)')
ax2.set_title('Portfolio Value Over Time')
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

In this notebook, you learned how to:
- ✓ Load real options data from DoltHub
- ✓ Construct an Iron Condor strategy
- ✓ Run a complete backtest with realistic data
- ✓ Analyze performance metrics and P&L attribution
- ✓ Visualize results and Greeks evolution

## Next Steps

- Try different strike selections (±3%, ±7%, etc.)
- Test on different time periods
- Experiment with different underlyings (AAPL, NVDA, etc.)
- Implement rolling strategies (monthly iron condors)
- Add adjustment rules (close early if profitable, etc.)